<a href="https://colab.research.google.com/github/Hemanthnaidu3737/Sentiment-Analysis-using-BERT-and-Transformer/blob/main/Copy_BERT_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch pandas scikit-learn flask-ngrok gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import gradio as gr



In [3]:
# Load dataset
data = pd.read_csv('/content/Train.csv')

# Check first few rows
print(data.head())

# Encode labels: Convert text labels into numbers
label_mapping = {'positive': 0, 'neutral': 1, 'negative': 2}
data['Sentiment'] = data['Sentiment'].map(label_mapping)



# Split dataset into train & validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['Product_Description'].tolist(), data['Sentiment'].tolist(), test_size=0.2, random_state=42
)



   Text_ID                                Product_Description  Product_Type  \
0     3057  The Web DesignerÛªs Guide to iOS (and Android...             9   
1     6254  RT @mention Line for iPad 2 is longer today th...             9   
2     8212  Crazy that Apple is opening a temporary store ...             9   
3     4422  The lesson from Google One Pass: In this digit...             9   
4     5526  RT @mention At the panel: &quot;Your mom has a...             9   

   Sentiment  
0          2  
1          2  
2          2  
3          2  
4          2  


In [4]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset objects
train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)


In [6]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        logits = model(**inputs).logits

    probabilities = F.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

    sentiment_labels = ["Positive", "Neutral", "Negative"]
    return sentiment_labels[predicted_class], probabilities.tolist()

def chatbot_response(user_input):
    sentiment, _ = predict_sentiment(user_input)

    response_map = {
        "Positive": "That's amazing! 😊 What made you happy today?",
        "Neutral": "Alright, anything specific on your mind? 🤔",
        "Negative": "I'm sorry to hear that. Do you want to talk about it? 😔"
    }

    return f"Sentiment: {sentiment}\nChatbot: {response_map.get(sentiment, 'I’m here to listen.')}"


In [7]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        logits = model(**inputs).logits

    probabilities = F.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

    sentiment_labels = ["Positive", "Neutral", "Negative"]
    return sentiment_labels[predicted_class], probabilities.tolist()

def chatbot_response(user_input):
    sentiment, _ = predict_sentiment(user_input)

    response_map = {
        "Positive": "That's amazing! 😊 What made you happy today?",
        "Neutral": "Alright, anything specific on your mind? 🤔",
        "Negative": "I'm sorry to hear that. Do you want to talk about it? 😔"
    }

    return f"Sentiment: {sentiment}\nChatbot: {response_map.get(sentiment, 'I’m here to listen.')}"


In [9]:
iface = gr.Interface(
    fn=chatbot_response,
    inputs="text",
    outputs="text",
    title="BERT Sentiment Analysis Chatbot",
    description="Type a message and the chatbot will analyze its sentiment and respond accordingly.",
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e1fe42ee9b6458a225.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
